# RoBERTa Office and General: 

In [1]:
# install transformers
!pip install transformers


Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [18]:
tf.random.set_seed(1234)

In [19]:
# load: 
import os 
import json
import numpy as np
import pandas as pd
import re
from tqdm import tqdm
import time
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

In [20]:
# load the pre-trained models: 
from transformers import RobertaTokenizer, TFRobertaModel
with tf.device('/device:GPU:0'):
    tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
    model = TFRobertaModel.from_pretrained("roberta-base")

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


In [21]:
# load csvs: 
# train_df = pd.read_csv('train_df_processed.csv')
# test_df =pd.read_csv('test_df_processed.csv')
train_path = '/home/ec2-user/SageMaker/Kartik_Amazon_Thesis/general_training_data'
rating_managment_explicit = pd.read_csv(os.path.join(train_path,'rating_managment_explicit_general.csv'))
rating_managment_explicit.columns

Index(['review_id', 'full_review', 'rating_managment_explicit',
       'rating_managment_explicit_ohe'],
      dtype='object')

In [22]:
rating_managment_explicit.rating_managment_explicit_ohe.value_counts()

0    11057
1      173
Name: rating_managment_explicit_ohe, dtype: int64

In [23]:
rating_managment_explicit[rating_managment_explicit['rating_managment_explicit_ohe']==1].iloc[19].full_review

'Not sure why there are so many positive ratings for this product because they do not do the job. Pen skips on all types of paper surfaces. Either these overwhelmingly positive reviews are phony or the manufacturer has reduced quality to make more money.'

In [24]:
rating_managment_explicit= rating_managment_explicit[['rating_managment_explicit_ohe', 'full_review']]

In [25]:
# train test split - split trainning into train and validation, take 10% as validation
train, test_df = train_test_split(rating_managment_explicit, test_size= 0.3, random_state=42, stratify=rating_managment_explicit['rating_managment_explicit_ohe'])

In [26]:
# train test split - split trainning into train and validation, take 10% as validation
train_df_1, val_df = train_test_split(train, test_size= 0.2, random_state=42, stratify=train['rating_managment_explicit_ohe'])

In [27]:
val_df.rating_managment_explicit_ohe.value_counts()

0    1549
1      24
Name: rating_managment_explicit_ohe, dtype: int64

In [28]:

class create_tensor_dataset(): 
    
    '''Purpose: Create a tensorflow dataset for model training
    df_to_convert: The dataframe of interest, that contains the 
    text columns
    text: The column within the dataframe containing the text 
    batch_size: int, batch size
    tokenizer: The tokenizer to be utilised
    labels: The class/label column within the dataframe
    '''
    
    def __init__(self, df_to_convert, text,target_col, batch_size, labels=None ): 
        self.df_to_convert = df_to_convert
        self.text = text # column containing the text (here chunked_data)
        self.target_col = target_col # y column of interest here label
        self.batch_size = batch_size
        self.tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
        self.labels = labels # list of all the labels 
        
    def generate_training_data(self, ids, masks):
        for i, text in tqdm(enumerate(self.df_to_convert[self.text])):
            with tf.device('/device:GPU:0'):
                
                tokenized_text =  self.tokenizer.encode_plus(
                    text,# The sequence or batch of sequences to be encoded.
                    max_length=256, #Controls the maximum length to use by one of the truncation/padding parameters.
                    truncation=True, # Truncate to a maximum length specified with the argument max_length
                    padding='max_length', #pad to a maximum length specified with the argument max_length 
                    add_special_tokens=True,# encode the sequences with the special tokens relative to their model.
                    return_tensors='tf'   #return TensorFlow tf.constant objects.
                )
                ids[i, :] = tokenized_text.input_ids ##List of token ids to be fed to a model
                masks[i, :] = tokenized_text.attention_mask #List of indices specifying which tokens should be attended to by the model
        return ids, masks
    
    def ohe_label(self): 
         # convert label values to a list 
        return self.df_to_convert[self.target_col].values.tolist()

    def DatasetMapFunction(self, input_ids, attn_masks, labels):
        # create and return key-value pair of input ids, attention mask and the corresnponding labels
        return {
            'input_ids': input_ids,
            'attention_mask': attn_masks
        }, labels
    
    def tensor_data(self): 
        # input_ids array
        X_input_ids = np.zeros((len(self.df_to_convert), 256))
        # attention mask array
        X_attn_masks = np.zeros((len(self.df_to_convert), 256))
        # retrieve the input ids and attention masks
        X_input_ids, X_attn_masks = self.generate_training_data(X_input_ids, X_attn_masks)
        # creation of tensorflow dataset: 
        dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, self.ohe_label()))
        dataset = dataset.map(self.DatasetMapFunction)
        dataset = dataset.shuffle(10000).batch(self.batch_size, drop_remainder=True)
        return dataset

In [29]:
# creation of tensorflow dataset for training
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)
train_tensor =create_tensor_dataset(train_df_1, 'full_review', 'rating_managment_explicit_ohe',8 )
train_dataset =train_tensor.tensor_data() 

6288it [00:07, 812.44it/s]


In [30]:
# validation tensor dataset: 
with tf.device('/device:GPU:0'):
    val_tensor =create_tensor_dataset(val_df, 'full_review', 'rating_managment_explicit_ohe', 8)
    val_dataset =val_tensor.tensor_data() 

1573it [00:02, 733.11it/s]


In [31]:

# defining 2 input layers for input_ids and attn_masks
with tf.device('/device:GPU:0'):
    input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
    attn_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')
    bert_embds = model.roberta(input_ids, attention_mask=attn_masks)[1] # pooled output
    # intermediate_layer_1 = tf.keras.layers.Dropout(0.5, name="dropout")(bert_embds)# 0 -> activation layer (3D), 1 -> pooled output layer (2D)
    intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer')(bert_embds)
    # intermediate_layer_2 = tf.keras.layers.Dropout(0.4, name="dropout2")(intermediate_layer)
    output_layer = tf.keras.layers.Dense(1, activation='sigmoid', name='output_layer')(intermediate_layer) # sigmoid -> calcs probs of binary classes
    model = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
    model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 roberta (TFRobertaMainLayer)   TFBaseModelOutputWi  124645632   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                         

In [32]:
# utilization of adam optimization 
optim = tf.keras.optimizers.Adam(learning_rate=1e-5)
# utilised binary cross entropy loss 
loss_func = tf.keras.losses.BinaryCrossentropy()
model.compile(optimizer=optim, loss=loss_func, metrics=['accuracy'])

# early stopping - administer val_loss: 
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping()
custom_early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience= 3,
    mode='min'
)

with tf.device('/device:GPU:0'):

    hist = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=100, 
        callbacks =[custom_early_stopping]
    )

Epoch 1/100
786/786 [==============================] - 396s 463ms/step - loss: 0.0823 - accuracy: 0.9844 - val_loss: 0.0615 - val_accuracy: 0.9821
Epoch 2/100
786/786 [==============================] - 355s 452ms/step - loss: 0.0496 - accuracy: 0.9862 - val_loss: 0.0458 - val_accuracy: 0.9821
Epoch 3/100
786/786 [==============================] - 351s 446ms/step - loss: 0.0333 - accuracy: 0.9916 - val_loss: 0.0492 - val_accuracy: 0.9847
Epoch 4/100
786/786 [==============================] - 350s 446ms/step - loss: 0.0232 - accuracy: 0.9946 - val_loss: 0.0515 - val_accuracy: 0.9841
Epoch 5/100
786/786 [==============================] - 351s 447ms/step - loss: 0.0158 - accuracy: 0.9963 - val_loss: 0.0656 - val_accuracy: 0.9866


In [33]:
def prepare_data(input_text, tokenizer):
    with tf.device('/device:GPU:0'):
        
        token = tokenizer.encode_plus(
            input_text,
            max_length=256, 
            truncation=True, 
            padding='max_length', 
            add_special_tokens=True,
            return_tensors='tf'
        )
        return {
            'input_ids': tf.cast(token.input_ids, tf.float64),
            'attention_mask': tf.cast(token.attention_mask, tf.float64)
        }

def make_predictions(model, input_text,threshold, label_list=None ): 
    with tf.device('/device:GPU:0'):
        processed_data = prepare_data(input_text, tokenizer)
        probs = model.predict(processed_data)
#         print(probs)
        if probs[0]> threshold: 
            return label_list[0]
        else: 
            return label_list[1]
#     return(probs[0])

In [34]:
# tt = test_df.head(20)
label_list= [1, 0]
test_df['result']= test_df.full_review.apply(lambda x: make_predictions(model, x, 0.5, label_list))

1/1 [==============================] - 0s 59ms/step


1/1 [==============================] - 0s 60ms/step


1/1 [==============================] - 0s 68ms/step


1/1 [==============================] - 0s 60ms/step


1/1 [==============================] - 0s 61ms/step


...

1/1 [==============================] - 0s 63ms/step


1/1 [==============================] - 0s 62ms/step


1/1 [==============================] - 0s 61ms/step


1/1 [==============================] - 0s 62ms/step


1/1 [==============================] - 0s 62ms/step


In [35]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
print (classification_report(test_df['rating_managment_explicit_ohe'], test_df['result']))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      3317
           1       0.69      0.35      0.46        52

    accuracy                           0.99      3369
   macro avg       0.84      0.67      0.73      3369
weighted avg       0.99      0.99      0.99      3369



In [36]:
model.save(os.path.join(os.getcwd(), "rating_managment_explicit_model_fnl"))

INFO:tensorflow:Assets written to: /home/ec2-user/SageMaker/Kartik_Amazon_Thesis/rating_management_explicit/rating_managment_explicit_model_fnl/assets


INFO:tensorflow:Assets written to: /home/ec2-user/SageMaker/Kartik_Amazon_Thesis/rating_management_explicit/rating_managment_explicit_model_fnl/assets


In [37]:
model = tf.keras.models.load_model( "rating_managment_explicit_model_fnl")